# The Shadow Angle

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/russellgoyder/sundial/HEAD?labpath=docs%2Fnb%2Fshadow_angle.ipynb)

Given the shadow plane and hour angle described in [The Hour Angle](hour_angle.md), we can calculate a unit vector in the dial face parallel to the shadow, and track its progress across the dial face.


In [3]:
from analemma.algebra import frame, render, result

## Intersecting the Shadow Plane and Dial Face

The shadow cast on the dial face by the gnomon is parallel to the intersection between the shadow bivector $S$ (the plane containing the sun ray and the gnomon) and $G$ (the dial face). We can construct [this intersection][analemma.algebra.result.dialface_shadowbivector_intersection] in Geometric Algebra (GA) as

$$u = \langle I G S \rangle_1$$

where $I = e_1 \wedge e_2 \wedge e_3$ is the pseudo-scalar (directed volume).

In [4]:
Sn = result.shadow_bivector_explicit()
Gn = frame.dialface()
u = result.dialface_shadowbivector_intersection(Gn, Sn)
render.align("u", u)

<IPython.core.display.Math object>

## Normalization

Want to normalize $u$ to get $\hat{u} \equiv \hat{w}$, a unit vector parallel to the shadow $w$. We could square and add the above components and attempt to factorize, but there is a route to a simpler expression because GA allows us to project two- and higher-dimensional objects onto each other just like vectors.

The cosine of the angle between $S$ and $G$ is given by

$$\cos(\Psi) = \frac{S\cdot G}{\sqrt{-S^2}\sqrt{-G^2}}$$

Given that $S^2 = -\sin^2(\Xi)$ and $G^2 = -1$, we have

$$ \cos(\Psi) = \frac{S\cdot G}{\sin(\Xi)}$$

We can calculate a simple [expression for $\cos(\Psi)$][analemma.algebra.result.dialface_shadowbivector_angle_cos]:

In [6]:
cosPsi = result.dialface_shadowbivector_angle_cos(Gn, Sn)
render.expression(r"\cos(\Psi)", cosPsi)

<IPython.core.display.Math object>

Now, the length of $u$ is related to $\Psi$ as follows.

$$u^2 = (I\; G \times S)^2 = (\frac{I}{2}(GS-SG))^2 = -\frac{1}{4}(GSGS+SGSG-2S^2G^2)$$

But

$$(S\cdot G)^2 = \frac{1}{4}(SG+GS)^2 = \frac{1}{4}(SGSG+GSGS+2S^2G^2)$$

So,

$$u^2 = S^2G^2 - (S\cdot G)^2 = \sin^2(\Xi) - (S\cdot G)^2 = \sin^2(\Xi)\sin^2(\Psi)$$

(my [Geometric Algebra cheat sheet](https://russellgoyder.github.io/geometric-algebra-cheat-sheet/) can be useful at times like this)

## The Unit Shadow

We can now form [$\hat{w}$][analemma.algebra.result.unit_shadow] by dividing $u$ by its length $\sin(\Xi)\sin(\Psi)$. Note that this length is not the length of the shadow vector $w$ which we will call $L$ such that $w = L \hat{w}$. It is related to $L$ as we shall see in [The Shadow Length](shadow_length.md).

In [9]:
w_hat = result.unit_shadow(Gn, Sn)
render.align(r"\hat{w} = \frac{w}{L} = \frac{u}{\sin(\Xi)\sin(\Psi)}", w_hat)


<IPython.core.display.Math object>

## The Shadow Angle Relative to Noon

Let's get the angular coordinate of the shadow relative to noon, call it $\zeta$. At [noon][analemma.algebra.result.unit_noon_shadow] we have $\mu = 0$:

In [10]:
render.expression(r"\hat{w}_{\mu=0}", result.unit_noon_shadow(Gn, Sn))

<IPython.core.display.Math object>

So $\cos(\zeta) = \hat{w}(\mu) \cdot \hat{w}(0)$ and the easiest route to $\sin(\zeta)$ is to use the fact that the shadow lives in the plane of the dial face, so $\hat{w}(\mu) \wedge \hat{w}(0) = \sin(\zeta) G$.

In [12]:
render.expressions(
    (r"\sin(\zeta)", r"\cos(\zeta)", r"\tan(\zeta)"),
    result.noon_angle_sincos(Gn, Sn) + (result.noon_angle_tan(Gn, Sn),)
    )

<IPython.core.display.Math object>

Next in [The Shadow Length](shadow_length.md), we calculate the length of the shadow, whose tip traces the analemma.